<a href="https://colab.research.google.com/github/HomayounfarM/Timeseries/blob/main/Timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/#create=true&language=r, or this short URL https://colab.to/r

In [ ]:
install.packages("googledrive")
library("googledrive")

#Classical Decomposition Model






In [ ]:
# load required libraries
library(Kendall)
library(tidyverse)
library(fpp2) 
library(forecast) 
library(tibbletime) 
library(tsbox) 
library(gridExtra) 
library(knitr) 
library(ggthemes)
library(Kendall)

In [ ]:
#install.packages('Kendall')
#install.packages('gridExtra')
#install.packages('fpp2')
#install.packages('tibbletime')
#install.packages('tsbox')
#install.packages('ggthemes')

Assuming we have the following time series:

In [ ]:
LakeHuron
class(LakeHuron)
str(LakeHuron)


# Convert time series to tibble object
LakeHuron_tbl <- data.frame(water_level=as.numeric(LakeHuron), 
                        year=as.numeric(time(LakeHuron)))

class(LakeHuron_tbl)

# Plot the data 
ggplot(LakeHuron_tbl,aes(x=year,y=water_level)) + 
  ggtitle("Lake Huron water levels (1875-1972)") + 
  geom_line() + geom_point() +  
  ylab("Water Level") + xlab("Year") 

In [ ]:
#Perform the Mann-Kendall Trend Test
MannKendall(LakeHuron)

The test statistic is -0.354 and the corresponding two-sided p-value is 2.4718e-07. Because this p-value is less than 0.05, we will reject the null hypothesis of the test and conclude that a trend is present in the data.

So indeed, we have some data from 1875 to 1972 on water levels for Lake Huron. What can we say about the trend? Let’s estimate a couple and see!

In [ ]:
huron_linear <- tslm(LakeHuron ~ trend) ## Fit linear trend 
huron_quad <- tslm(LakeHuron ~ trend + I(trend^2)) ## Fit quadratic trend 
huron_ma7 <- ma(LakeHuron, order=7) # Fit a q=3 moving average (ma=2q+1)

# Put all of these together
LakeHuron_with_fits <- cbind(LakeHuron, 
                             Linear_trend=fitted(huron_linear), 
                             Quadratic_trend=fitted(huron_quad),
                             Moving_avg7 = huron_ma7) 
                             
# Construct the plot 
autoplot(LakeHuron_with_fits) + 
  ylab("Water Level (in feet)") + xlab("Year") +
  ggtitle("Lake Huron water levels (1875-1972)") +  
  guides(colour= guide_legend(title = "Data series")) + 
  scale_colour_manual(values=c("black","red","blue","yellow"))  

Now, let's look at the residuals.
Remember that after removing the trend, the result should somehow look like a stationary process or at least like a mean-zero process!

In [ ]:
# Extract residuals for each estimated trend
LakeHuron_resids<-cbind(Res_orig=LakeHuron-mean(LakeHuron),
                        Res_linear=LakeHuron-fitted(huron_linear),
                        Res_quad=LakeHuron-fitted(huron_quad), 
                        Res_MA7 = LakeHuron - huron_ma7)

# produce the autoplot 
autoplot(LakeHuron_resids, facet=TRUE) + xlab("Year") + ylab("Residuals") +  
  ggtitle("Lake Huron water levels (1875-1972) Residuals") + 
  geom_hline(yintercept = 0) + 
  guides(colour=guide_legend(title="Data Series"))

We can see that for all trends, they definitely look a lot more mean-zero than the original one, and the MA7 (order-7 moving-average) looks a bit more like White Noise than the others. You might also notice that we lost some data along the process: indeed, as we can only go over a window [-q,q], we will lose a couple of observations at the borders. We can inspect this more formally using the ACF plots we had seen before:

In [ ]:
ggAcf(LakeHuron) + ggtitle("ACF for original series") + ylim(c(-1,1))
ggAcf(LakeHuron_resids[,"Res_linear"]) + ggtitle("ACF removing linear trend") + ylim(c(-1,1)) 
ggAcf(LakeHuron_resids[,"Res_quad"]) + ggtitle("ACF removing quadratic trend") + ylim(c(-1,1)) 
ggAcf(LakeHuron_resids[,"Res_MA7"]) + ggtitle("ACF removing MA7 trend")+ ylim(c(-1,1))

Indeed, all of our trend estimations do significantly much better than the original series, as more lags fall within the confidence bounds. However, we see that sometimes over-estimating the trend can lead to further correlation. Overall, the MA7 seems to be the best choice. The drawback is that by employing an MA filter, we sacrifice some data, and therefore, some predictive power. In the end, it is up to you to make these choices over which trend estimation to use over another!

https://medium.com/analytics-vidhya/a-complete-introduction-to-time-series-analysis-with-r-classical-decomposition-model-a4548a0c99b9

#ARIMA
ARIMA is an acronym for Auto Regressive (AR) Integrated (I) Moving Average (MA) which indicates that an ARIMA model has three components to it.

ARIMA models can be expressed in two forms: Non-seasonal models where the model exhibits an order in the form of (p,d,q) where:

p = The order of the Auto Regressive Model

d = The order of differencing

q = The order of the Moving Average

###Auto Regressive Models (AR):

Auto regressive models are similar to a regression model but the regressor in this case is the same dependent variable with a specific lag.

###Differencing (I):

For ARIMA to perform at its best it needs the data to be stationary. That means that the mean and variance are constant over the entire set. Differencing is used to transform the data so that it is stationary.

###Moving Average (MA):

Moving averages are widely used in time series analysis and is an already well-known concept. It involves getting the average of the points in a series for a specific lag.

###Seasonal ARIMA models (SARIMA):

These models take into account the seasonality in the data and does the same ARIMA steps but on the seasonal pattern. So, if the data has a seasonal pattern every quarter then the SARIMA will get an order for (p,d,q) for all the points and a (P,D,Q) for each quarter.

###Regression Models VS. ARIMA:

Now comes the real deal. ARIMA models are great for forecasting blindly into the future using historical data. However, sometimes we don’t need to forecast blindly, sometimes we have variables that can help us predict future behavior. Regression models are ideal in this scenario.